# Sesion 3: Taller de tecnicas de modelado computacional de fluidos

## Elementos Finitos: Metodo Galerkin Discontinuo

**Por: Erick Urquilla, Universidad de Tennessee, Knoxville, USA**

Resolveremos las ecuaciones que gobiernan la dinámica de fluidos poco profundos incompresibles. El fluido está limitado inferiormente y a los costados por superficies rígidas y superiormente sin restricción. Estas ecuaciones se derivan de las ecuaciones de Navier-Stokes cuando el dominio espacial horizontal del fluido excede significativamente al vertical, manteniendo el equilibrio hidrostático y densidad constante:

$$
\frac{\partial h}{\partial t} + \frac{\partial}{\partial x}\left(hu\right) = 0
$$
$$
\frac{\partial}{\partial t}\left(hu\right) + \frac{\partial}{\partial x}\left(hu^2 + \frac{gh^2}{2}\right) = 0
$$

Aquí, $h$ es la altura del fluido, $u$ es la velocidad horizontal y $g$ es la aceleración de la gravedad. El movimiento del fluido está dictado exclusivamente por las ecuaciones de conservación de masa y momento.

## Paquetes utilizados

En este proyecto, utilizaremos varios paquetes de Python que nos ayudarán a realizar diferentes tareas de manera eficiente. A continuación, se describen los paquetes que se utilizarán:

1. **numPy**
2. **matplotlib**
3. **IPython**
4. **time**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
import time

## Scripts utilizados

En esta sesion importaremos los siguientes scripts

1. **bases.py**
2. **galerkin_discontinuo.py**
3. **paso_de_tiempo.py**
4. **graficos.py**

Puedes encontrar estos scripts en la misma ruta que este notebook

In [ ]:
# !rm -r 1D-Shallow-Water-Equation
# !git clone --branch curso_unah_sesion_3 --single-branch https://github.com/erickurquilla1999/1D-Shallow-Water-Equation.git
# !mv 1D-Shallow-Water-Equation/*.py ./

import bases
import galerkin_discontinuo 
import paso_de_tiempo
import graficos

## Parámetros de entrada

1. Dominio espacial de la simulación
2. Número de elementos y número de nodos interiores por elemento
3. Tiempo de la simulación y número de pasos
4. Número de puntos en la cuadratura de Gauss para integración numérica

In [ ]:
# Parametros de entrada

# Dominio espacial de la simulacion 
x_inicial = 0 # (m) cooordenda inicial del dominio
x_final = 10 # (m) coordenada final del dominio

# Parametros del metodo de elementos finitos
N_elementos = 6 # numero de elementos
N_nodos = 4 # numero de nodos por elemento (por simplicidad solo consideramos N_nodos >= 2)

# Dominio temporal 
n_pasos = 100 # numero de pasos temporales
t_total = 1 # (s) tiempo final

# Integracion numerica con cuadratura de Gauss
n_nodos_cuadratura_gauss = 20

# Variables utiles
longitud_elemento = (x_final - x_inicial) / N_elementos

## Generacion de malla

La generación de una malla es fundamental en el método de elementos finitos. La malla divide el dominio espacial en elementos finitos más pequeños, lo que permite aproximar la solución de la ecuación diferencial en cada uno de estos elementos. En el método Galerkin Discontinuo, cada elemento puede tener su propia solución polinómica, lo que permite capturar discontinuidades y variaciones abruptas en las variables del fluido.

**Ejercicio 1:** Dadas las coordenadas inicial `x_inicial`, coordenada final `x_final`, el número de elementos `N_elementos` y el número de nodos por elemento `N_nodos`. Codifica una malla cartesiana unidimensional con las coordenadas de los nodos interiores de cada elemento (elementos y nodos igualmente espaciados en una malla regular). El numpy array resultado del ejercicio debe llamarse `malla`. La forma del array `malla` debe ser `(N_elementos, N_nodos)`. Es decir, el componente `malla[i,j]` representa la coordenada en `x` del nodo `j` del elemento `i`. 

Como ejemplo, observa la siguiente imagen de una `malla` con `N_elementos=3` y `N_nodos=4`. El arreglo `malla` contiene las posiciones de los nodos interiores de cada elemento.

![Malla](imagenes/malla.png)

In [ ]:
print(f'Generando malla \nDominio físico: [{x_inicial},{x_final}] metros\nNúmero de elementos: {N_elementos}\nNodos por elemento: {N_nodos}\n')

# Generar coordenadas de los elementos y nodos en el espacio físico
malla = np.zeros((N_elementos, N_nodos)) # nunpy array que almacenara las coordenadas de los nodos de cada elemento

#-----------------------------------------------------------------------------------------
# Escribe tu solucion al ejercicio 1 a continuacion ...
malla = np.array([np.linspace(x_inicial + i * (x_final - x_inicial) / N_elementos, x_inicial + (i + 1) * (x_final - x_inicial) / N_elementos, N_nodos) for i in range(N_elementos)])      
#-----------------------------------------------------------------------------------------

## Condiciones Iniciales

Las condiciones iniciales del fluido están dadas por:

$$h_0 = 1 + 0.1 e^{-(x-5)^2}$$
$$u_0 = 0$$

Las fronteras del fluido están ubicadas en $x = 0$ y $x = 10$ metros.

**Ejercicio 2:** Crea dos arrays de numpy que contengan las condiciones iniciales:

1. El primer array debe llamarse `h` y tener la forma `(N_elementos, N_nodos)`. `h[i,j]` representará la altura en metros del fluido en el nodo $j$ del elemento $i$.
2. El segundo array debe llamarse `u` y tener la forma `(N_elementos, N_nodos)`. `u[i,j]` representará la velocidad en metros por segundo del fluido en el nodo $j$ del elemento $i$.

In [ ]:
# Generando condiciones iniciales
h = np.zeros((N_elementos, N_nodos)) # Height (cm)
u = np.zeros((N_elementos, N_nodos)) # Velocity (cm/s)

#-----------------------------------------------------------------------------------------
# Escribe tu solucion al ejercicio 2 a continuacion ...
h = 1.0 + 0.1 * np.exp( - ( malla - 5.0 )**2 ) # Height (cm)
u = malla * 0.0 # Velocity (cm/s)
#-----------------------------------------------------------------------------------------

## Aproximación funcional de la solución: Polinomios de Lagrange

Para crear una solución funcional a la ecuación de agua profunda, usaremos los polinomios de Lagrange definidos como
$$
\phi_i(x) = \prod_{\substack{0 \leq m \leq p \\ m \neq i}} \frac{x - x_m}{x_i - x_m}.
$$
Aquí, $x_i$ es la posición del nodo $i$. 

Los polinomios de Lagrange tienen la siguiente propiedad:
$$
\phi_i(x_j) = \delta_{ij}.
$$
Utilizando los polinomios de Lagrange, una solución funcional en el elemento $k$ puede escribirse como
$$
u^{k}(t,x) = \sum_{i=0}^{N} U^{k}_{i}(t)\phi_{i}^k(x),
$$
donde $N$ es el número de nodos en el elemento $k$.

**Ejercicio 3:** Escriba una función que, dados los nodos de un elemento, el índice $i$ del polinomio de Lagrange y una posición $x$, retorne el polinomio de Lagrange $i$ evaluado en $x$.


In [ ]:
def polinomios_de_lagrange(nodos, i, x):
    '''
    Dado un conjunto de nodos en un elemento esta funcion
    evalua el polinomio de Lagrange i (es decir el polinomio de 
    lagrange que es uno cuando es evaluado en la posicion del nodo i) 
    en x la posicion x

    Parámetros:
        nodos (numpy.ndarray): Array de nodos de un elemento de la malla.
            ejemplo: nodos = np.array([x0, x1, ..., xN]) donde N es el numero de nodos por elemento
        i (int): Índice del nodo en el cual el polinomio de lagrange es uno.
            ejemplo: i = 1
        x (float): Punto en el cual se evalúa la función base.
            ejemplo: x = 0.5

    Retorna:
        float: Valor de la función base de Lagrange i en el punto x.
    '''
    
    # inicializa el polinomio de lagrange i evaluado en x
    polinomio_de_lagrange_i_evaluado_en_x = 1.0

    #-----------------------------------------------------------------------------------------
    # Escribe tu solucion al ejercicio 3 a continuacion ...
    
    # calcula el polinomio de lagrange i evaluado en x
    for j in range(N_nodos):
        # si j es diferente de i
        if j != i:
            # multiplica el polinomio de lagrange i evaluado en x por el factor correspondiente
            polinomio_de_lagrange_i_evaluado_en_x *= (x - nodos[j]) / (nodos[i] - nodos[j])

    #-----------------------------------------------------------------------------------------

    return polinomio_de_lagrange_i_evaluado_en_x

# Verifica que tu solucion satisface la propiedad de los polinomios de lagrange 
# \phi_i(x_j) = \delta_{ij}
nodos_prueba = malla[0]
i_prueba = 1
x_prueba = 0
phi_prueba = polinomios_de_lagrange(nodos_prueba, i_prueba, x_prueba)
print(f"Para los nodos {nodos_prueba}, el polinomio de lagrange i={i_prueba} evaluado en x={x_prueba} es {phi_prueba}")

## Generando cuadratura y pesos de Gauss para integracion numerica

La cuadratura de Gauss es un método para aproximar la integral de una función utilizando una suma ponderada de los valores de la función en puntos específicos dentro del dominio de integración. La fórmula general para la cuadratura de Gauss es:

$$
\int_a^b f(x) \, dx \approx \frac{b - a}{2} \sum_{i=1}^n w_i \, f\left( x_i \right)
$$
$$
x_i = \frac{b - a}{2} \xi_i + \frac{a + b}{2}
$$
donde:
- $\xi_i$ es el nodos de Gauss $i$ and $\xi_i\in[-1,1]$.
- $w_i$ son los pesos asociados a cada punto de evaluación.
- $n$ es el número de puntos de evaluación.

Los puntos $x_i$ y los pesos $w_i$ se eligen de tal manera que el método sea exacto para polinomios de grado $2n-1$ o menor. Estos puntos y pesos se derivan de los polinomios de Legendre, que son ortogonales en el intervalo $[-1, 1]$.

En la siguiente linea de codigo obtendremos la cuadratura de Gauss y los pesos de Gauss.

In [ ]:
cuadratura_de_gauss, pesos_de_gauss = np.polynomial.legendre.leggauss(n_nodos_cuadratura_gauss)

## La siguiente de codigo calcula:

2. Evalua los polinomios de lagrange en la cuadratura de Gauss
3. Evalua las derivada en $x$ de los polinomios de lagrange en la cuadratura de Gauss

In [ ]:
polinomios_de_lagrange_en_cuadratura_de_gauss, derivada_x_polinomios_de_lagrange_en_cuadratura_de_gauss = bases.generate_reference_space(malla, cuadratura_de_gauss, polinomios_de_lagrange)

## Calculo de la matriz de masa

En esta secion calcularemos la matriz de masa definida como
$$
M_{ij} = \int _{x_0}^{x_{N-1}} \phi_{i}(x)\phi_{j}(x)\,dx.
$$
Para calcular la intengral usaremos el metodo de cuadratura de Gauss
$$
M_{ij} = \frac{L}{2} \sum_{m=0}^{n-1} w_i \, \phi_{i}(x_m)\phi_{j}(x_m),
$$
aqui $L = x_{N-1}-x_0$ es la longitud de los elementos y $n$ es el numero de nodos en la cuadratura de Gauss

Ejercicio 4: Calcule la matriz de masa utilizando las formulas mostradas anteriormente.

In [ ]:

def calcula_inversa_matriz_de_masa(longitud_elemento_, pesos_de_gauss_, polinomios_de_lagrange_en_cuadratura_de_gauss_):
    """
    Calcula la inversa de la matriz de masa para un elemento finito unidimensional.

    Parametros:

    longitud_elemento_ (float): La longitud del elemento finito.
        ejemplo: longitud_elemento = 1.0

    pesos_de_gauss_ (numpy.ndarray): Los pesos de Gauss utilizados en la cuadratura de Gauss.
        ejemplo: pesos_de_gauss = [w0, w1, ..., wN] donde N es el numero de nodos de cuadratura de Gauss
    
    polinomios_de_lagrange_en_cuadratura_de_gauss_ (numpy.ndarray): Los valores de los polinomios de Lagrange evaluados en los puntos de cuadratura de Gauss.
        ejemplo: 
            polinomios_de_lagrange_en_cuadratura_de_gauss_ = [[phi_0(x_0), phi_0(x_1), ..., phi_0(x_N)], [phi_1(x_0), phi_1(x_1), ..., phi_1(x_N)], ..., [phi_N(x_0), phi_N(x_1), ..., phi_N(x_N)]]
            polinomios_de_lagrange_en_cuadratura_de_gauss_[i] es un array con los valores de phi_i(x) evaluados en los nodos de cuadratura de Gauss.
            polinomios_de_lagrange_en_cuadratura_de_gauss_[i] = [phi_i(x_0), phi_i(x_1), ..., phi_i(x_N)]

    Retorna:
    
    numpy.ndarray: La inversa de la matriz de masa.
    """

    # Inicializa la matriz de masa
    matriz_de_masa = np.zeros((N_nodos, N_nodos))

    #-----------------------------------------------------------------------------------------
    # Escribe tu solucion al ejercicio 4 a continuacion ...

    # Calcula la matriz de masa
    for i in range(N_nodos):
        for j in range(N_nodos):
            # Calcula la integral de phi_i(x) * phi_j(x) dx
            matriz_de_masa[i, j] = 0.5 * longitud_elemento_ * np.sum(pesos_de_gauss_ * polinomios_de_lagrange_en_cuadratura_de_gauss_[i] * polinomios_de_lagrange_en_cuadratura_de_gauss_[j])
    # Calcula la inversa de la matriz de masa
    inversa_matriz_de_masa = np.linalg.inv(matriz_de_masa)

    #-----------------------------------------------------------------------------------------

    return inversa_matriz_de_masa

# calcula la matriz de masa inversa, esto es la inversa de la matriz: M_ij = integral phi_i(x) phi_j(x) dx
matriz_de_masa_inversa = calcula_inversa_matriz_de_masa(longitud_elemento, pesos_de_gauss, polinomios_de_lagrange_en_cuadratura_de_gauss)

## Calculo de la matriz de rigidez

En esta seccion se calcula la matriz de rigidez definida como
$$
S_{ij} = \int _{x_0}^{x_{N-1}} \frac{d\phi_{i}(x)}{dx}\phi_{j}(x)\,dx.
$$

In [ ]:
# calcula la matriz de rigidez S_ij = integral (dphi_i(x)/dx)phi_j(x) dx
matriz_de_rigidez = galerkin_discontinuo.calcula_matrix_de_rigidez(longitud_elemento, pesos_de_gauss, polinomios_de_lagrange_en_cuadratura_de_gauss, derivada_x_polinomios_de_lagrange_en_cuadratura_de_gauss)

## Evolucion temporal de la solucion

El método de Euler hacia adelante es un método numérico simple y explícito para resolver ecuaciones diferenciales ordinarias (ODEs). Se basa en la aproximación de la derivada de una función en un punto utilizando diferencias finitas. La fórmula general del método de Euler hacia adelante es:

$$
y_{n+1} = y_n + \Delta t \cdot \frac{dy(t_n)}{dt}
$$

donde:
- $y_{n+1}$ es el valor de la solución en el siguiente paso de tiempo.
- $y_n$ es el valor de la solución en el paso de tiempo actual.
- $\Delta t$ es el tamaño del paso de tiempo.
- $f(t_n, y_n)$ es la función que describe la derivada de $y$ con respecto al tiempo en el paso de tiempo actual.

Este método es fácil de implementar y computacionalmente eficiente, pero puede ser inestable si el tamaño del paso de tiempo no es lo suficientemente pequeño.

En nuestro caso $d\vec{U}^k_i(t)/dt$ vendra dada por 
$$
\frac{d\vec{U}^k_i(t)}{dt} = \sum_{j=0}^{N-1} M^{-1}_{ij}\vec{R}_j
$$

**Ejercicio 5**: Escriba el metodo Euler hacia adelante para evaluar la evolucion temporal de $\vec{U}^k_i$. Esto es
$$
\vec{U}^k_{i}(t_{n+1}) = \vec{U}^k_{i}(t_{n}) + \Delta t \frac{d\vec{U}^k_i(t_n)}{dt}
$$

In [ ]:
# time step
time_step = np.array(t_total/n_pasos) 

# evolving in time the PDE
for number_of_t_step in np.arange(n_pasos):

    # calculando el vector residual
    vector_residual_1, vector_residual_2 = paso_de_tiempo.calcular_vector_residual(h, u, matriz_de_rigidez)

    # -----------------------------------------------------------------------------------------
    # Escribe tu solucion al ejercicio 5 a continuacion ...

    # inicializando los valores de d1U/dt y d2U/dt
    d1U_dt = np.zeros((N_elementos, N_nodos))
    d2U_dt = np.zeros((N_elementos, N_nodos))

    # calculando la derivada temporal de 1U y 2U ...
    # ... Escribe aqui d1U_dt y d2U_dt ... 
    # ... Haz un loop sobre cada elemento ...
    # ... Utiliza la matriz de masa inversa y el vector residual en cada elemento para calcular las derivadas ...
    for i in range(N_elementos):
        d1U_dt[i] = matriz_de_masa_inversa @ vector_residual_1[i]
        d2U_dt[i] = matriz_de_masa_inversa @ vector_residual_2[i]

    # inicializando los valores de dh/dt y du/dt
    dh_dt = np.zeros((N_elementos, N_nodos))
    du_dt = np.zeros((N_elementos, N_nodos))

    # calculando la derivada temporal de h y u
    # ... Escribe aqui dh_dt y du_dt ...
    # ... recuerda que 1U=h y 2U=hu por lo que du_dt = ( d2U_dt - u * dh_dt ) / h ...
    dh_dt = d1U_dt
    du_dt = np.where( h == 0 , 0 , ( d2U_dt - u * dh_dt ) / h )

    # inicializando los valores de h y u en el siguiente paso de tiempo
    h_nuevo = np.zeros((N_elementos, N_nodos))
    u_nuevo = np.zeros((N_elementos, N_nodos))

    # calculando los valores de h y u en el siguiente paso de tiempo con el metodo de Euler explicito
    # ... Escribe aqui h_nuevo y u_nuevo dados por el metodo de Euler explicito ...
    h_nuevo = h + dh_dt * time_step
    u_nuevo = u + du_dt * time_step
    
    # -----------------------------------------------------------------------------------------

    # graficando la simulacion
    graficos.plot_simulation(malla, h_nuevo, u_nuevo, N_elementos, time_step, number_of_t_step)

    # actualizando los valores de h y u
    h = h_nuevo
    u = u_nuevo

print(f'Done')